In [3]:
data_filename_train = 'data/all_gene_annotations.added_incompleteness_and_contamination.training.tsv'
data_filename_test = "data/all_gene_annotations.added_incompleteness_and_contamination.testing.tsv"
y_filename = "data/bacdive_scrape_20230315.json.parsed.anaerobe_vs_aerobe.with_cyanos.csv"


In [4]:
from utils import read_xy_data

d3_train, X_train, y_train = read_xy_data(data_filename_train, y_filename)
num_aerobs_train = y_train['oxytolerance'].sum()
num_anaerobs_train = len(y_train['oxytolerance']) - num_aerobs_train


print(f"Total number of genomes for training = {num_aerobs_train+num_anaerobs_train}")
print(f"Percentage of aerobs in train dataset {round(num_aerobs_train/(num_aerobs_train + num_anaerobs_train), 2)}")

y name = data/bacdive_scrape_20230315.json.parsed.anaerobe_vs_aerobe.with_cyanos.csv
Counts of each class in training/test data: shape: (2, 2)
┌──────────────┬───────┐
│ oxytolerance ┆ len   │
│ ---          ┆ ---   │
│ str          ┆ u32   │
╞══════════════╪═══════╡
│ aerobe       ┆ 58356 │
│ anaerobe     ┆ 29808 │
└──────────────┴───────┘
Total number of genomes for training = 88164
Percentage of aerobs in train dataset 0.66


In [5]:
d3_test, X_test, y_test = read_xy_data(data_filename_test, y_filename)
num_aerobs_test = y_test['oxytolerance'].sum()
num_anaerobs_test = len(y_test['oxytolerance']) - num_aerobs_test

print(f"Total number of genomes for testing = {num_aerobs_test+num_anaerobs_test}")
print(f"Percentage of aerobs in test dataset {round(num_aerobs_test/(num_aerobs_test + num_anaerobs_test), 2)}")

y name = data/bacdive_scrape_20230315.json.parsed.anaerobe_vs_aerobe.with_cyanos.csv
Counts of each class in training/test data: shape: (2, 2)
┌──────────────┬───────┐
│ oxytolerance ┆ len   │
│ ---          ┆ ---   │
│ str          ┆ u32   │
╞══════════════╪═══════╡
│ aerobe       ┆ 17460 │
│ anaerobe     ┆ 8172  │
└──────────────┴───────┘
Total number of genomes for testing = 25632
Percentage of aerobs in test dataset 0.68


In [6]:
import random 
import polars as pl


random.seed(42)

def table_row_subsampling(d3):

    target_column = "oxytolerance"

    d_gtdb = d3.to_pandas()
    X = d3.select(pl.exclude([target_column,'phylum','class','order','family','genus'])).to_pandas()
    

    # Map oxytolerance to 0, 1, 2
    if 'anaerobic_with_respiration_genes' in d3['oxytolerance'].to_list():
        classes_map = {
            'anaerobe': 0,
            'aerobe': 1,
            'anaerobic_with_respiration_genes': 2,
        }
    else:
        classes_map = {
            'anaerobe': 0,
            'aerobe': 1,
        }
    
    # Generate y vector with 0s, and 1s
    y = d3.select(
        pl.when(pl.col(target_column) == 'anaerobe').then(0)
        .when(pl.col(target_column) == 'aerobe').then(1)
        .when(pl.col(target_column) == 'anaerobic_with_respiration_genes').then(2)
        .otherwise(None)  # Handle cases not in the map
        .alias(target_column)
    )
    y = y.to_pandas()


    
    num_aerobs = y['oxytolerance'].sum()
    num_anaerobs = len(y['oxytolerance']) - num_aerobs
    
    
    # Sub-sampling training data
    indices_aerobs = y[y['oxytolerance'] == 1].index.tolist()
    indices_anaerobs = y[y['oxytolerance'] == 0].index.tolist()

    if num_aerobs > num_anaerobs:
        print(f"Sub-sampling {num_aerobs} aerobs to {num_anaerobs} anaerobs")
        subsampled_aerobs = random.sample(indices_aerobs, num_anaerobs)
        final_row_indices = subsampled_aerobs + indices_anaerobs
    else:
        print(f"Sub-sampling {indices_anaerobs} aerobs to {num_aerobs} anaerobs")
        subsampled_anaerobs = random.sample(indices_anaerobs, num_aerobs)
        final_row_indices = subsampled_anaerobs + indices_aerobs

    X_subsampled = X.iloc[final_row_indices].reset_index(drop=True)
    y_subsampled = y.iloc[final_row_indices].reset_index(drop=True)

    print(f"Sub-sampled table length = {len(y_subsampled)} with { y_subsampled['oxytolerance'].sum()} aerobs and  {len(y_subsampled['oxytolerance']) - y_subsampled['oxytolerance'].sum()} anaerobs")
    
    return X_subsampled, y_subsampled

In [7]:
X_subsampled_train, y_subsampled_train = table_row_subsampling(d3_train)

Sub-sampling 58356 aerobs to 29808 anaerobs
Sub-sampled table length = 59616 with 29808 aerobs and  29808 anaerobs


In [8]:
X_subsampled_test, y_subsampled_test = table_row_subsampling(d3_test)

Sub-sampling 17460 aerobs to 8172 anaerobs
Sub-sampled table length = 16344 with 8172 aerobs and  8172 anaerobs


In [9]:
import pandas as pd

df = pd.DataFrame(X_subsampled_train)
filename = 'data/all_gene_annotations.added_incompleteness_and_contamination.subsampled.training.tsv'
df.to_csv(filename, sep="\t", index=False)

df = pd.DataFrame(X_subsampled_test)
filename = 'data/all_gene_annotations.added_incompleteness_and_contamination.subsampled.testing.tsv'
df.to_csv(filename, sep="\t", index=False)


In [10]:
X_subsampled_test_val = X_subsampled_test.drop(['accession','false_negative_rate',	'false_positive_rate'], axis = 1).values

X_subsampled_test_val_scaled = []
for row in X_subsampled_test_val:
    row_sm = float(sum(row))
    X_subsampled_test_val_scaled.append([float(row_i)/row_sm for row_i in row])


In [ ]:
print(X_subsampled_test_val)
X_subsampled_train_val = X_subsampled_train.drop(['accession','false_negative_rate',	'false_positive_rate'], axis = 1).values


X_subsampled_train_val_scaled = []
for row in X_subsampled_train_val:
    row_sm = float(sum(row))
    X_subsampled_train_val_scaled.append([float(row_i)/row_sm for row_i in row])

X_subsampled_train_val_scaled = np.array(X_subsampled_train_val_scaled)
    

[[1 2 0 ... 0 0 0]
 [1 4 4 ... 1 2 0]
 [0 1 1 ... 0 0 0]
 ...
 [2 1 0 ... 0 1 0]
 [1 1 0 ... 2 0 0]
 [0 0 1 ... 1 2 0]]


In [ ]:
X_subsampled_train_val

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap



# Create a custom colormap
colors = ListedColormap(["tab:blue", "tab:red"])


pca = PCA(n_components=20)

#print(X_test_reduc_dimens.shape)

X_train_val = X_subsampled_train_val#.values
y_train_val = y_subsampled_train.values

X_train_pca = pca.fit_transform(X_train_val)

print(f"Data after PCA reduction: {X_train_pca.shape}")

plt.figure()

color = ["tab:blue", "tab:red"]

plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train_val, alpha=0.3, s = 1, cmap=colors)


explained_variance_ratio = pca.explained_variance_ratio_

print("Explained variance ratio:", explained_variance_ratio)
print("Total explained variance:", sum(explained_variance_ratio))


In [38]:
from sklearn.manifold import TSNE
import numpy as np

np.random.seed(42)


# Initialize and apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=70, learning_rate=5, max_iter=1000)#, init='pca')
X_tsne = tsne.fit_transform(X_train_val) #init = 'pca'

print(X_tsne.shape)

# Visualize the t-SNE output
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_train_val, alpha=0.7, s = 2, cmap=colors)

AttributeError: 'list' object has no attribute 'shape'

In [10]:
#print(X_subsampled_train)
import numpy as np

def remove_low_var_cogs(X):
 #   print(X)
    var_arr = []
    names= []
    for column in X.columns:
        if "COG" in column:
            names.append(column)
            column_array = X[column].values
            var_arr.append(np.var(column_array))
    
    var_thresh = 3
    ind_filter_var = [i for i in range(len(var_arr)) if var_arr[i] > var_thresh]
    print(len(var_arr))
    print(len(ind_filter_var))
    
    #df_subsampled = df.iloc[:, columns_to_subsample]
    names_filt = [names[i] for i in ind_filter_var ]
    
  #  table_indices = [ind+3 for ind in ind_filter_var]

  #  print(table_indices[-1])
   # print( X.iloc[:, table_indices[-1]])
    # table_indices = [0, 1, 2] + table_indices
    
    # X_subsampled_train
    
    X_reduc_dimens = X.iloc[:, ind_filter_var]#table_indices]
    return X_reduc_dimens

In [11]:
print(f"Before dim. reduction {X_test.shape}")

X_test_reduc_dimens = remove_low_var_cogs(X_test)

print(f"After dim. reduction {X_test_reduc_dimens.shape}")

Before dim. reduction (25632, 2677)
2677
56
After dim. reduction (25632, 56)
